<img align="right" src="https://ds-cs-images.s3.ap-northeast-2.amazonaws.com/Codestates_Fulllogo_Color.png" width=100>

## *DATA SCIENCE / SECTION 2 / SPRINT 1 / NOTE 4*

# 📝 Assignment
---

# Logistic Regression

이번에는 [**Cardiovascular Disease dataset**](https://www.kaggle.com/sulianova/cardiovascular-disease-dataset) 데이터셋을 사용해 봅시다. 데이터에 대한 설명을 캐글 페이지에서 잘 살펴보시고 과제를 진행해 주세요!

### 1) 데이터를 전처리 하겠습니다.
- 데이터를 불러오세요.(read_csv 사용)
- id 칼럼은 drop하세요.
- missing value를 확인하세요.
- 중복된 샘플이 있는지 확인하세요.
- **중복된 샘플을 제거하고 남은 샘플의 수를 과제 제출폼에 제출하세요.**

In [ ]:
import pandas as pd
df = pd.read_csv('https://ds-lecture-data.s3.ap-northeast-2.amazonaws.com/cardio/cardio_train.csv', sep=';')

df.head()

,id,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
0,0,18393,2,168,62.0,110,80,1,1,0,0,1,0
1,1,20228,1,156,85.0,140,90,3,1,0,0,1,1
2,2,18857,1,165,64.0,130,70,3,1,0,0,0,1
3,3,17623,2,169,82.0,150,100,1,1,0,0,1,1
4,4,17474,1,156,56.0,100,60,1,1,0,0,0,0


In [ ]:
import pandas as pd
df = pd.read_csv('https://ds-lecture-data.s3.ap-northeast-2.amazonaws.com/cardio/cardio_train.csv', sep=';')

df= df.drop(columns= 'id')
df.isnull().sum(axis = 1) #missing value = 0

df.duplicated()
df = df.drop_duplicates()

df.shape  #(69976, 12)

(69976, 12)

### 2) 다운로드 한 훈련 데이터를 훈련/검증/테스트 세트로 나누어 주세요.
- 타겟 특성을 확인하고 타겟 변수를 나누어 줍니다.
- 훈련과 테스트 데이터로 나누어 줍니다(train_test_split 사용, train/test = 80/20, random_state=2)
- 훈련데이터를 훈련과 검증데이터로 나누어 줍니다(train_test_split 사용, train/validation = 80/20, random_state=2)
- **검증 데이터(X_val)의 갯수를 과제 제출폼에 제출하세요.**

In [ ]:
from sklearn.model_selection import train_test_split
#target
target = 'cardio'
features = ['age'	,'gender'	,'height'	,'weight'	,'ap_hi'	,'ap_lo'	,'cholesterol',	'gluc',	'smoke',	'alco',	'active']
X= df[features]
y= df[target]



X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, train_size=0.8, random_state=2)
X_val

,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active
36174,20787,1,168,67.0,120,80,3,3,0,0,1
4101,20578,2,176,80.0,160,80,1,1,1,1,1
19924,18368,2,174,78.0,120,80,2,1,0,0,1
41995,17415,2,177,62.0,120,80,1,1,1,0,1
39939,23303,2,168,98.0,140,90,3,3,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...
67206,16823,1,155,63.0,120,80,1,1,0,0,0
25036,22517,1,141,60.0,150,100,3,3,0,0,0
12153,15245,2,165,64.0,120,80,1,1,0,0,1
25375,18260,2,170,70.0,120,80,3,3,0,0,1


### 3) 사이킷런의 LogisticRegression을 사용해 모델을 만들어 보세요.
- 모델을 만들기 전에 기준모델을 설정합니다.
- **LogisticRegression 모델을 학습하고(max_iter=1000) 검증세트 정확도를 과제 제출폼에 제출하세요.**


In [ ]:
y_train.value_counts(normalize=True)
# mode(): Return the highest frequency value in a Series.
major = y_train.mode()[0]

# 타겟 샘플 수 만큼 0이 담긴 리스트를 만듭니다. 기준모델로 예측
y_pred = [major] * len(y_train)

from sklearn.metrics import accuracy_score
print("training accuracy: ", accuracy_score(y_train, y_pred))

# 검증세트 에서도 정확도를 확인해 볼 수 있습니다.
y_pred = [major] * len(y_val)
print("validation accuracy: ", accuracy_score(y_val, y_pred))



training accuracy:  0.501697034655234
validation accuracy:  0.49821364773133264


In [ ]:
from sklearn.linear_model import LogisticRegression

logistic = LogisticRegression(max_iter=1000)
logistic.fit(X_train, y_train)

print('검증세트 정확도', logistic.score(X_val, y_val))

검증세트 정확도 0.7073061807788495


### 4) 데이터에서 범주형 성질을 지닌 특성을 찾아 원핫인코딩 하세요.
- category_encoders를 사용해서 'gender' 특성만 원핫인코딩 하세요.
- LogisticRegression 모델을 다시 학습합니다.
- **검증세트 정확도를 과제 제출폼에 제출하세요.**

In [ ]:
!pip install --upgrade category_encoders

Requirement already up-to-date: category_encoders in /usr/local/lib/python3.7/dist-packages (2.2.2)


In [ ]:
from category_encoders import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler


#X_train['gender'] = X_train['gender'].replace([1,2],['F','M'])
#X_test['gender'] = X_test['gender'].replace([1,2],['F','M'])
#X_val['gender'] = X_val['gender'].replace([1,2],['F','M'])
def gender_change(num):
  if type(num) != str:
    if num ==1:
      num='women'
    else:
      num='men'
  else:
      return num
  return num


X_train['gender']=X_train['gender'].apply(gender_change)
X_val['gender']=X_val['gender'].apply(gender_change)
X_test['gender']=X_test['gender'].apply(gender_change)




encoder = OneHotEncoder(use_cat_names=True)
X_train= encoder.fit_transform(X_train)
X_val = encoder.transform(X_val)
X_train.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


,age,gender_women,gender_men,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active
221,17591,1,0,157,70.0,160,100,3,1,0,0,0
59211,19677,0,1,174,80.0,150,80,1,1,0,0,1
26654,23504,1,0,158,76.0,140,90,3,3,0,0,1
33421,18036,0,1,173,128.0,160,110,3,1,0,1,0
12046,20225,0,1,176,109.0,120,80,1,1,1,1,1


In [ ]:
logistic = LogisticRegression(max_iter=1000)
logistic.fit(X_train, y_train)

print('검증세트 정확도', logistic.score(X_val, y_val))

검증세트 정확도 0.7093604858878171


###

### 5) 학습한 모델을 검증세트에서 최적화 시킨 후, 테스트세트에서 딱 한 번 정확도를 확인해 보세요.
- **테스트 세트 정확도를 과제 제출폼에 제출하세요.(0.712 이상)**

In [ ]:
X_test['gender']=X_test['gender'].apply(gender_change)
X_test = encoder.transform(X_test)

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
logistic_scaled = LogisticRegression(max_iter=1000)
logistic_scaled.fit(X_train_scaled, y_train)

print('검증세트 정확도', logistic_scaled.score(X_val_scaled, y_val))

검증세트 정확도 0.723383351196856


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
X_test_scaled = scaler.transform(X_test)
print('테스트 세트 정확도', logistic_scaled.score(X_test_scaled, y_test))


테스트 세트 정확도 0.7264218348099457


## 🔥 도전과제:

### 6) EDA를 추가적으로 수행하고 가능한 방법으로 모델을 튜닝해서 결과를 만들어 보세요. 최소한 다음 내용들이 포함이 되어야 합니다.
- 새로운 특성을 최소한 한 개 이상 만드세요.
- 특성들을 확인해서 아웃라이어라고 생각되는 데이터를 가진 샘플을 제거하세요.
- [feature scaling](https://scikit-learn.org/stable/modules/preprocessing.html) 을 사용하세요. (여러가지 종류의 Scalar를 어떤 상황에 적용해야 하는지 알아보고 사용해 보세요.)
- 만들어진 모델 성능을 리포팅하고 해석합니다.
